![Julia logo](https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/fa84a1c62df35176760ff42f3c4862cc632b01e6/images/julia-logo-dark.svg)

Julia is a fast, easy-to-write, easy-to-read language that really excels at number crunching.

There's a decent (if kind of dense and fast-moving) Wikibook titled [Introducing Julia](https://en.wikibooks.org/wiki/Introducing_Julia), which isn't a bad place to start.  The [Julia home page](https://julialang.org/) also has some decent documentation and links to numerous tutorials.

# Why Julia?

Fast!

Solves the two-language problem (or three-language depending on your field).

Lots of cool features.

Vibrant community.

Rapidly gaining popularity.

# Why not Julia?

Young.  The package ecosystems still lag behind Python and R for a lot of things, but this is rapidly changing.

It's a big language.  There is a lot of syntax and a lot of ways to do things.  It can be a steep initial learning curve.

Do Python and R do what you need?  Then there's no compelling _need_ to learn Julia--except for the fact that it's cool, and that learning new languages generally broadens your knowledge and skills as a programmer.

Rapid development and change, especially in the third party packages.  This can mean a lot of breaking changes happen between releases, but this has really settled down a lot in recent years (but it's still more of an issue than with Python or R).

# The future is "Julia, R, and Python"

...in fact, the "Ju" part of Jupyter stands for Julia.  Julia is excellent at solving a set of problems that Python and R really struggle with; it is likely to become a dominant language _alongside_ them, and I don't see it replacing either language at any point in the forseeable future.

Let's just write some code and see the basics.

In [1]:
# Code to calculate the length of a Collatz Sequence given the starting value
function collatz_len(n)
    seqlen = 0
    while n > 1
        if n % 2 == 0
            # div() is integer division; `/` always does floating point division
            n = div(n, 2)
        elseif n % 2 == 1
            # [number][variable] = [number] * [variable]--it's multiplication, just like
            # in math!
            n = 3n + 1
        end
        seqlen += 1
    end
    
    return seqlen
end

@time maximum(collatz_len.(1:1_000_000))

  0.136716 seconds (2 allocations: 7.629 MiB)


524

In [6]:
x = [1, 2, 3, 4, 5]
x[1]

1

Some major and obvious differences from Python:

- Defining functions uses `function foo() [code] end`, not `def foo(): [code, ends when indent resets]`.  Julia doesn't use indentation for anything syntactic--it uses `end` to mark the end of a code block (e.g. a function definition, a loop, a conditional).  *You* should still use indentation though to make your code readable!
- `elseif` insead of `elif`
- Using `/` for division will always return a float, which can cause some issues for this particular code when the inputs get really large, so `div()` is used to do integer division.  (integers in, integers out).
- `@macro` is a _syntactic macro._  This is a really hard to explain idea, but it's basically like a function that takes in _code_--not values--as its arguments, manipulates that code, and returns the new code.
    - This it not just manipulating input _text_; it actually works on the _code itself._
    - We aren't going to worry about writing our own macros.
- `.` syntax for mapping/broadcasting.  `foo.(x)` automatically vectorized `foo` over the input `x`; Julia does this for you for every function in the language.
    - For infix operatirs, it's `.+`--the `.` comes before the operator, not after it.  So: `x .+ y` is element-wise addition.
- Ranges are created using the `:` operator, and include the upper bound.  (Python's `range()` function excludes the upper bound).
- Speed.  Compare the following to the Python equivalent in the companion notebook.  The Python version is ~100x slower.
    - Yes, we could speed the Python code up using some tricks like `@numba.jit`, but Julia is fast _out of the box._

Some rapid-fire Julia:

In [10]:
# functions that are single statements can be defined in a single line:
function add1(x)
    return x + 1
end

println(add1(10))

# is equivalent to
add1(x) = x + 1
println(add1(10))

11
11


Julia supports--and encourages!--using Unicode symbols in your code.  Some built-in functions have unicode aliases:

In [11]:
println(sqrt(2))
println(√2)

println(π)

☿ = 2π
println(☿)

1.4142135623730951
1.4142135623730951
π
6.283185307179586


To type these characters: use a backslash, followed by the LaTeX command for the character, then hit "tab".  This requires your editor to be set up with Julia support, though.

Julia is a _very_ function-oriented language, and it supports both piping and function composition out of the box:

In [12]:
# the following are equivalent
foo(x) = sqrt(log(x))
println(foo(10))

# the ∘ symbol (`\circ`) is the _function composition_ operator.
# (f ∘ g) creates a new function that's identical to f(g(...)).
foo(x) = (sqrt ∘ log)(x)
println(foo(10))

# this is called a "point-free" solution: we define a function just
# in terms of other functions, but no variables.  This is, I think,
# a very elegant and beautiful way to define functions.
point_free = (sqrt ∘ log)
println(point_free(10))

# `|>` is the pipe operator.
println(10 |> log |> sqrt)

1.5174271293851465
1.5174271293851465
1.5174271293851465
1.5174271293851465


# Multiple Dispatch

Get ready to have your minds blown.  Julia takes this idea from R and Haskell, but improves on it: you can have multiple different functions with the same name, and Julia figures out which one to call based on the types and number of all the arguments.  E.g.:

In [13]:
my_function(x::Int64) = x + 10
my_function(x::String) = "$x is a string."

println(my_function(10))
println(my_function("10"))

20
10 is a string.


The `::Int64` syntax is how Julia does type annotations.  `x::Int64` means "`x` should be of type `Int64` (a 64-bit integer)."  Whenever you call a function in Julia, it looks at the types of every single argument, and finds whichever definition 1) uses the name of the function you're calling, and 2) is the most specific match to the values you're passing.

Each definition of a function is called a _method._  Julia doesn't have objects and classes like Python does, so you'll never see `x.something()` in idiomatic Julia code.  `x.something()` is usually called a "method" in languages that support it, but since Julia doesn't have this, it re-uses the word "method" to refer to "different bits of code assigned to the same function name."

_Digression:_ this is actually the same usage as in Python, but it requires digging deeper into what `x.something()` does.  A method in Python is, conceptually, no different from saying "`something(x)`, but throw an error if `x` isn't the right type."  Different classes can each implement their own version of `something()`, so really, each definition is kind of just different code, given the same name, and which instance of the code gets called depends on the type of `x`.  (_end of digression_)

In the above example, we first call `my_function(10)`.  `10` gets interpreted by Julia as an `Int64` (64-bit integer); in general, "integer literals" like this get interpreted as whichever integer is the default for your specific hardware.  Nowadays, on desktops and laptops, this is all but guaranteed to be 64 bit integers, but it could be 32 bits on older hardware, or some single-board computers like Raspberry Pis.  Julia looks for a definition of `my_function` that takes an `Int64` as its first argument; it finds one, so it runs that.

In the next line, we pass `"10"`, which is a string; Julia looks for a definition of `my_function` that takes a string as its input, finds one, and calls it.

But wait!  It gets better!  Julia not only dispatches on the type of the first argument, but on the types of _all_ arguments--_and on the number of arguments._

In [14]:
# this looks like it's redefining the function, but it's adding a new method.
my_function(x, y) = "You gave me two arguments but I don't know what to do with them."
my_function(x::Number) = "$x is a number, but not an Int64, so I don't know what to do with it."
my_function(x::Number, y::Number) = x + y

println(my_function("hello", 10)) # calls the method for my_function(x, y)
println(my_function(10.1))
println(my_function(10.0, π))

You gave me two arguments but I don't know what to do with them.
10.1 is a number, but not an Int64, so I don't know what to do with it.
13.141592653589793


The first line of the above cell--`my_function(x, y) = ...`, is actually a shorthand for: `my_function(x::Any, y::Any)`, where `Any` is a special type that means "it can be any type.  Literally any.  It does not matter."

`Number` is a special kind of type called an _abstract type._  This is a concept that's easier to explain visually:

![Julia's numeric type hierarchy](https://upload.wikimedia.org/wikipedia/commons/d/d9/Julia-number-type-hierarchy.svg)

Individual values--like variables--can be only be assigned types that are at the very bottom of this tree.  So I can tell Julia "`x` is an Int32", but I can't tell Julia "`x` is a Real."  But, when defining a function, you can say "the first argument needs to be a Real," and Julia will interpret that as: "the first argument needs to be any concrete type that's underneath 'Real' in the above image."  So in the previous cell, `my_function(x::Number, y::Number)` gets called whenever `x` ia any sub-type of `Number`, `y` is any sub-type of `Number`, _and_ there are no methods for `my_function()` that 1) have more specific type annotations, and 2) also match the types of x and y.

You can define your own new types--both concrete and abstract--and choose where they fit in the Julia hierarchy.  We aren't going to worry about defining new abstract types tonight.

A quick example that highlights the "most specific method gets called" behavior of Julia:

In [15]:
# Definition 1
example(x::Number) = x + 10

# Definition 2
example(x::Integer) = "You passed an integer."

# Definition 3
example(x::Int8) = "That's an 8-bit integer!"

# This gets interpreted as an Int64.  That matches definitions 1 and 2, but not 3.
# Definition 2 is more specific--Integer is lower down in the tree than Number--so
# definition 2 gets called.
println(example(10))

You passed an integer.


In [28]:
# This does not match any of the above definitions.  There is no definition
# of example(x::Any), so Julia will instead throw an error.
println(example("10"))

LoadError: MethodError: no method matching example(::String)
[0mClosest candidates are:
[0m  example([91m::Int8[39m) at In[27]:8
[0m  example([91m::Integer[39m) at In[27]:5
[0m  example([91m::Number[39m) at In[27]:2

# More functions: optional and keyword arguments

Unlike Python, Julia makes a very strict distinction between positional and keyword arguments, and a separate distinction between optional and required.

In [39]:
function argument_example(
        required_position,
        optional_positional=10
        ;                      # all remaining arguments are keyword
        required_keyword,
        optional_keyword=-10,
        )
    return Dict(
        "Required Postional" => required_position,
        "Required Keyword" => required_keyword,
        "Optional Positional" => optional_positional,
        "Optional Keyword" => optional_keyword,
    )
end

println("All defaults")
argument_example(100, required_keyword=8)

All defaults


Dict{String, Int64} with 4 entries:
  "Required Postional"  => 100
  "Optional Positional" => 10
  "Required Keyword"    => 8
  "Optional Keyword"    => -10

In [40]:
# Can't pass positionally-defined arguments as keywords.
argument_example(10, optional_positional=9, required_keyword=8)

LoadError: MethodError: no method matching argument_example(::Int64, ::Int64; optional_positional=9, required_keyword=8)
[0mClosest candidates are:
[0m  argument_example(::Any, ::Any; required_keyword, optional_keyword) at In[39]:1[91m got unsupported keyword argument "optional_positional"[39m
[0m  argument_example(::Any) at In[39]:1[91m got unsupported keyword arguments "optional_positional", "required_keyword"[39m

In [39]:
# Can't pass keyword-defined arguments positionally.
argument_example(10, 9, 8)

LoadError: MethodError: no method matching argument_example(::Int64, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  argument_example(::Any, ::Any; required_keyword, optional_keyword) at In[37]:1
[0m  argument_example(::Any) at In[37]:1

This takes some getting used to coming from Python, but you'll adjust pretty quick. Besides: Julia will encourage you to write shorter functions, which tend to take fewer arguments anyways.  Python is happy to encourage large functions with dozens of arguments (granted, that's an extreme case and one you should avoid unless you have good reason).

# Everything is functions

In Julia, like in R, _everything is functions._  You never see `x.something()`; it's always `something(x)`.  This idea is extremely pervasive.  In fact, even infix operators like `+`, `*`, etc are actually functions:

In [52]:
println(5 + 5)
println(+(5, 5))

10
10


# Strings and Characters

Julia, unlike Python, makes a distinction between _character_ and _string_ types.  A character is a single, well, character, and characters are written with single quotes.  A string is an array of characters, and is written with double quotes.  _These are different data types!_

In [41]:
println(typeof('c'))
println(typeof("c"))

Char
String


In [42]:
'oops'

LoadError: syntax: character literal contains multiple characters

Usually, it's safe to just use strings for everything.

String interpolation--Julia's equivalent of Python's f-strings--is pretty simple:

In [45]:
x = 10

# just put x in a string with $x
println("x = $x")

# or, put a full Julia expression in the string with $(expr)
println("$x^2 = $(x^2)")

x = 10
10^2 = 100


In [47]:
my_string = "Hello world"
println(match(r"He..o", my_string))

RegexMatch("Hello")


Note that Julia uses `^`, not `**`, for exponentiation.

# Core data types: Arrays

Arrays are at the heart of Julia.  Arrays are _type-heterogeneous ordered collections,_ and they may be multi-dimensional. (basically think Numpy arrays).

In [47]:
my_array = [1, 2, 3, 4, 5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

Note that `Vector{Int64}` means "a Vector containing values of type Int64."  In Julia, "Vector" is identical to "one-dimensional array."  (It's actually an alias for one-dimensional array types).

Of course, since Julia has typeclasses, we could have `Vector{Number}`, which would mean "a one-dimensional array of values that can be any subtype of `Number`."

In [48]:
my_array = Number[1, 2, 3.3]

3-element Vector{Number}:
 1
 2
 3.3

In [49]:
# without specifying the type like that, Julia will coerce everything to be the same type.
[1, 2, 3.3]

3-element Vector{Float64}:
 1.0
 2.0
 3.3

_This type stuff matters._  If an array is all one type--and that type is a concrete type, like Int64--Julia can get a lot of extra speed when doing things to that array.  If the concrete types can be mixed, like in a `Vector{Number}`, Julia won't be able to run operations as quickly over that array.  It'll still be very fast by human standards, but wherever possible, use the same data types for all array elements.

There is a lot of syntax around arrays--creating, reshaping, initializing empty arrays, etc; we're not going to go into too much detail of that syntax here, except to see 2d arrays:

In [51]:
my_2d_array = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

Note the lack of commas!  `[1, 2, 3]` is a _column vector_; `[1 2 3]` is a _row vector_.  `;`, in array constructors, separates rows.

# Other useful types: Dictionaries and Sets

Dictionaries are basically the same as in Python:

In [48]:
my_dict = Dict(
    "Key" => "Value",
    10 => 100,
    π => "pi",
    "pi" => π,
)

println(my_dict[π])

pi


In [49]:
# add values via assignment
my_dict["e"] = 2.718
println(my_dict["e"])

2.718


In [50]:
# Remember: everything is functions, so keys(my_dict), not my_dict.keys(), like you'd
# do in Python
println(keys(my_dict))
println(values(my_dict))

Any[π, 10, "e", "Key", "pi"]
Any["pi", 100, 2.718, "Value", π]


Similarly, Sets are basically the same as in Python:

In [51]:
println(Set([1, 1, 2, 2, 3, 3]))

Set([2, 3, 1])


# Modules and Packages

Julia has two keywords that do the same broad thing as Python's `import`: `using` and, well, `import`.  The difference is confusing at first, but is pretty simple:
- `using` is the general import statement you want to use.  Unlike Python, everything from a module is by default imported into your program's global namespace; this allows modules to add new methods to functions or operators like `+`.
- `import` is used when you want to add new methods to some module's existing functions.

In [52]:
mean([1,2,3])

LoadError: UndefVarError: mean not defined

In [53]:
using Statistics

# mean() is from the Statistics module
println(mean([1,2,3]))

# We can also do this, though, in the rare cases we need to be super unambiguous
println(Statistics.mean([1,2,3]))

2.0
2.0


In [54]:
# but we can't add new methods:
mean(x::String) = "You can't take the mean of a string, silly!"

LoadError: error in method definition: function Statistics.mean must be explicitly imported to be extended

In [55]:
# to do that, we need to use `import [module]: [thing we want to add a new method to]`
import Statistics: mean
mean(x::String) = "You can't take the mean of a string, silly!"
mean("string")

"You can't take the mean of a string, silly!"

In [64]:
# we could use * to import everything from a module, but this is kind of bad form
import Statistics: *

So use `using` if you just need to be, well, `using` stuff from a module.  If you need to add new methods, use `import [module]: thing1, thing2, ...`.

# Structs

Julia lets you define your own concrete types using Structs.  These are structs almost exactly as they appear in languages like C, if you're familiar with those.  In the Python world, these are most similar to dataclasses or named tuples.  A struct is just a thing that contains a fixed set of values, each with potentially known types.

In [16]:
struct Card
    # the field names do need to be on their own lines
    # nop type annotation --> implicitly ::Any --> weaker
    # speed/performance guarantees
    value
    suit
end

# create a new Card instance by passing values positionally
king♡ = Card("king", "♡") # \heartsuit for the heart suit symbol
king♡

Card("king", "♡")

Access the fields of a struct using `struct.field` syntax.  Structs are by default immutable; they can't be changed after they're created.

In [17]:
println(king♡.suit)

♡


In [18]:
king♡.suit = "diamonds"

LoadError: setfield!: immutable struct of type Card cannot be changed

To allow this, define a struct using `mutable struct` instead of just `struct`.  This gives up a little bit of speed, but rarely enough to matter except when you're writing _extremely_ performance-critical code.  Also note that you can't re-define a struct without restarting Julia, so below, I'll make a differently named struct.

In [19]:
mutable struct MutableCard
    value::Union{String, Int64} # Union type: "either string or int64"
    suit::String
end
king♡ = MutableCard("king", "♡")
println(king♡.suit)
king♡.suit = "♢" # diamonds
println(king♡.suit)

♡
♢


In [23]:
struct Die
    nsides::Int64
end

import Base: *
*(n::Int64, d::Die) = sum(rand(1:d.nsides) for i ∈ 1:n)
d6 = Die(6)

Die(6)

In [37]:
5d6

16

# The REPL

Julia has an awesome REPL.  Which I will now show you by switching to a different window where I have it running.